In [6]:
import pandas as pd
import numpy as np
import cv2
import os
from ultralytics import YOLO

In [7]:
model = YOLO('../models/yolo/yolov8m.pt')

In [8]:
import cv2
import numpy as np
from datetime import datetime
from torchvision import transforms
from PIL import Image
import os

class PredPipe():
    def __init__(self, model, image_path, noise, blur):
        self.model = model
        self.image_path = image_path
        self.image = cv2.imread(image_path)
        self.name = image_path.split('/')[-1].split('.')[0]
        self.results = {}
        self.model_scores = []
        self.noise_and_blur = {
            'noise_pctg': noise,
            'blur_pctg': blur
        }
        self.timestamp = str(int(datetime.timestamp(datetime.now())))

    def add_noise_and_blur(self, save=False):
        image = cv2.imread(self.image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image.astype(np.float32) / 255.0

        noise = np.random.uniform(low=0.0, high=1.0, size=image.shape)
        noise_mask = np.random.binomial(1, self.noise_and_blur['noise_pctg'], size=image.shape[:-1])
        noise_mask = np.expand_dims(noise_mask, axis=-1)
        noise_mask = np.repeat(noise_mask, 3, axis=-1)
        image = noise_mask * noise + (1 - noise_mask) * image
        image = (image * 255.0).astype(np.uint8)
        noisy_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if save:
            noisy_image_path = f"../data/processed/noisy/{self.name}_{str(self.noise_and_blur['noise_pctg'])}_noisy_{self.timestamp}.jpg"
            cv2.imwrite(noisy_image_path, noisy_image)

        return image, noisy_image_path

    def save_traffic_lights(self, image, result, image_path, timestamp):
        output_folder = "../data/processed/outputs/semafaros/standard/"
        traffic_lights = []  # Lista para salvar os semáforos detectados

        for box in result.boxes:
            class_id = result.names[box.cls[0].item()]
            if class_id == "traffic light":
                cords = box.xyxy[0].tolist()
                cords = [int(round(x)) for x in cords]

                # Ajuste as coordenadas para garantir que estejam dentro dos limites da imagem
                cords[0] = max(0, cords[0])
                cords[1] = max(0, cords[1])
                cords[2] = min(image.shape[1], cords[2])
                cords[3] = min(image.shape[0], cords[3])

                traffic_light_image = image[cords[1]:cords[3], cords[0]:cords[2]]

                if traffic_light_image.shape[0] > 0 and traffic_light_image.shape[1] > 0:
                    # Certifique-se de que a região recortada não é vazia
                    traffic_lights.append(traffic_light_image)

        if not traffic_lights:
            raise ValueError("Nenhum semáforo detectado na imagem.")

        # Nome da foto original
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        
        for i, traffic_light_image in enumerate(traffic_lights):
            # Se houver mais de um semáforo na mesma foto, inclua um índice entre parênteses
            if i > 0:
                traffic_light_filename = f"traffic_light_{image_name}({i}).jpg"
            else:
                traffic_light_filename = f"traffic_light_{image_name}.jpg"
            
            traffic_light_path = os.path.join(output_folder, traffic_light_filename)
            
            # Verifique se o arquivo já existe antes de salvá-lo
            if not os.path.exists(traffic_light_path):
                cv2.imwrite(traffic_light_path, traffic_light_image)
                print(f"Semáforo {i + 1} da imagem {image_name} salvo em {traffic_light_path}")
            else:
                print(f"Semáforo {i + 1} da imagem {image_name} já existe em {traffic_light_path}. Não foi salvo novamente.")

    def save_cars(self, image, result, image_path, timestamp):
        output_folder = "../data/processed/outputs/carros/standard/"
        cars = []  # Lista para salvar os carros detectados

        for box in result.boxes:
            class_id = result.names[box.cls[0].item()]
            if class_id == "car":
                cords = box.xyxy[0].tolist()
                cords = [int(round(x)) for x in cords]

                # Ajuste as coordenadas para garantir que estejam dentro dos limites da imagem
                cords[0] = max(0, cords[0])
                cords[1] = max(0, cords[1])
                cords[2] = min(image.shape[1], cords[2])
                cords[3] = min(image.shape[0], cords[3])

                car_image = image[cords[1]:cords[3], cords[0]:cords[2]]

                if car_image.shape[0] > 0 and car_image.shape[1] > 0:
                    # Certifique-se de que a região recortada não é vazia
                    cars.append(car_image)

        if not cars:
            raise ValueError("Nenhum carro detectado na imagem.")

        # Nome da foto original
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        
        for i, car_image in enumerate(cars):
            # Se houver mais de um carro na mesma foto, inclua um índice entre parênteses
            if i > 0:
                car_filename = f"car_{image_name}({i}).jpg"
            else:
                car_filename = f"car_{image_name}.jpg"
            
            car_path = os.path.join(output_folder, car_filename)
            
            # Verifique se o arquivo já existe antes de salvá-lo
            if not os.path.exists(car_path):
                cv2.imwrite(car_path, car_image)
                print(f"Carro {i + 1} da imagem {image_name} salvo em {car_path}")
            else:
                print(f"Carro {i + 1} da imagem {image_name} já existe em {car_path}. Não foi salvo novamente.")

    def save_trucks(self, image, result, image_path, timestamp):
        output_folder = "../data/processed/outputs/caminhao/standard/"
        trucks = []  # Lista para salvar os caminhões detectados

        for box in result.boxes:
            class_id = result.names[box.cls[0].item()]
            if class_id == "truck":
                cords = box.xyxy[0].tolist()
                cords = [int(round(x)) for x in cords]

                # Ajuste as coordenadas para garantir que estejam dentro dos limites da imagem
                cords[0] = max(0, cords[0])
                cords[1] = max(0, cords[1])
                cords[2] = min(image.shape[1], cords[2])
                cords[3] = min(image.shape[0], cords[3])

                truck_image = image[cords[1]:cords[3], cords[0]:cords[2]]

                if truck_image.shape[0] > 0 and truck_image.shape[1] > 0:
                    # Certifique-se de que a região recortada não é vazia
                    trucks.append(truck_image)

        if not trucks:
            raise ValueError("Nenhum caminhão detectado na imagem.")

        # Nome da foto original
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        
        for i, truck_image in enumerate(trucks):
            # Se houver mais de um caminhão na mesma foto, inclua um índice entre parênteses
            if i > 0:
                truck_filename = f"truck_{image_name}({i}).jpg"
            else:
                truck_filename = f"truck_{image_name}.jpg"
            
            truck_path = os.path.join(output_folder, truck_filename)
            
            # Verifique se o arquivo já existe antes de salvá-lo
            if not os.path.exists(truck_path):
                cv2.imwrite(truck_path, truck_image)
                print(f"Caminhão {i + 1} da imagem {image_name} salvo em {truck_path}")
            else:
                print(f"Caminhão {i + 1} da imagem {image_name} já existe em {truck_path}. Não foi salvo novamente.")

    def save_people(self, image, result, image_path, timestamp):
        output_folder = "../data/processed/outputs/pessoas/standard/"
        people = []  # Lista para salvar as pessoas detectadas

        for box in result.boxes:
            class_id = result.names[box.cls[0].item()]
            if class_id == "person":
                cords = box.xyxy[0].tolist()
                cords = [int(round(x)) for x in cords]

                # Ajuste as coordenadas para garantir que estejam dentro dos limites da imagem
                cords[0] = max(0, cords[0])
                cords[1] = max(0, cords[1])
                cords[2] = min(image.shape[1], cords[2])
                cords[3] = min(image.shape[0], cords[3])

                person_image = image[cords[1]:cords[3], cords[0]:cords[2]]

                if person_image.shape[0] > 0 and person_image.shape[1] > 0:
                    # Certifique-se de que a região recortada não é vazia
                    people.append(person_image)

        if not people:
            raise ValueError("Nenhuma pessoa detectada na imagem.")

        # Nome da foto original
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        
        for i, person_image in enumerate(people):
            # Se houver mais de uma pessoa na mesma foto, inclua um índice entre parênteses
            if i > 0:
                person_filename = f"person_{image_name}({i}).jpg"
            else:
                person_filename = f"person_{image_name}.jpg"
            
            person_path = os.path.join(output_folder, person_filename)
            
            # Verifique se o arquivo já existe antes de salvá-lo
            if not os.path.exists(person_path):
                cv2.imwrite(person_path, person_image)
                print(f"Pessoa {i + 1} da imagem {image_name} salva em {person_path}")
            else:
                print(f"Pessoa {i + 1} da imagem {image_name} já existe em {person_path}. Não foi salva novamente.")

    def save_motorcycles(self, image, result, image_path, timestamp):
        output_folder = "../data/processed/outputs/motos/standard/"
        motorcycles = []  # Lista para salvar as motos detectadas

        for box in result.boxes:
            class_id = result.names[box.cls[0].item()]
            if class_id == "motorcycle":
                cords = box.xyxy[0].tolist()
                cords = [int(round(x)) for x in cords]

                # Ajuste as coordenadas para garantir que estejam dentro dos limites da imagem
                cords[0] = max(0, cords[0])
                cords[1] = max(0, cords[1])
                cords[2] = min(image.shape[1], cords[2])
                cords[3] = min(image.shape[0], cords[3])

                motorcycle_image = image[cords[1]:cords[3], cords[0]:cords[2]]

                if motorcycle_image.shape[0] > 0 and motorcycle_image.shape[1] > 0:
                    # Certifique-se de que a região recortada não é vazia
                    motorcycles.append(motorcycle_image)

        if not motorcycles:
            raise ValueError("Nenhuma moto detectada na imagem.")

        # Nome da foto original
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        
        for i, motorcycle_image in enumerate(motorcycles):
            # Se houver mais de uma moto na mesma foto, inclua um índice entre parênteses
            if i > 0:
                motorcycle_filename = f"motorcycle_{image_name}({i}).jpg"
            else:
                motorcycle_filename = f"motorcycle_{image_name}.jpg"
            
            motorcycle_path = os.path.join(output_folder, motorcycle_filename)
            
            # Verifique se o arquivo já existe antes de salvá-lo
            if not os.path.exists(motorcycle_path):
                cv2.imwrite(motorcycle_path, motorcycle_image)
                print(f"Moto {i + 1} da imagem {image_name} salva em {motorcycle_path}")
            else:
                print(f"Moto {i + 1} da imagem {image_name} já existe em {motorcycle_path}. Não foi salva novamente.")

    def predict_image(self, add_noise=False, save=True):
        if add_noise:
            image, noisy_image_path = self.add_noise_and_blur(save)
            results = self.model.predict(noisy_image_path)
            result = results[0]
            if save:
                predicted_image_path = f"../data/processed/outputs/{self.name}_{str(self.noise_and_blur['noise_pctg'])}_noisy_predicted_{self.timestamp}.jpg"
        else:
            results = self.model.predict(self.image_path)
            result = results[0]
            if save:
                predicted_image_path = f"../data/processed/outputs/{self.name}_predicted_{self.timestamp}.jpg"

        img = cv2.imread(self.image_path) if not add_noise else cv2.imread(noisy_image_path)

        for box in result.boxes:
            class_id = result.names[box.cls[0].item()]
            cords = box.xyxy[0].tolist()
            cords = [int(round(x)) for x in cords]
            conf = round(box.conf[0].item(), 2)

            # Desenhe um rótulo com confiança, mas sem bordas verdes
            label = f"{class_id} {conf}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            font_thickness = 1
            font_color = (0, 0, 255)
            background_color = (0, 0, 0)
            label_size, baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
            label_origin = (cords[0], cords[1] - label_size[1])
            cv2.rectangle(img, (cords[0], cords[1]), (cords[0] + label_size[0], cords[1] - label_size[1]), background_color, -1)
            cv2.putText(img, label, label_origin, font, font_scale, font_color, font_thickness)

        if save:
            cv2.imwrite(predicted_image_path, img)

        self.save_traffic_lights(img, result, self.image_path, self.timestamp)
        self.save_cars(img, result, self.image_path, self.timestamp)
        self.save_trucks(img, result, self.image_path, self.timestamp)
        self.save_people(img, result, self.image_path, self.timestamp)
        self.save_motorcycles(img, result, self.image_path, self.timestamp)

        return img, self.results


In [9]:
image_directory = '../data/frames/images'

for filename in os.listdir(image_directory):
    if filename.endswith("_original.jpg"):
        image_path = os.path.join(image_directory, filename)
        pred = PredPipe(model, image_path, 0, 0)
        
        try:
            # Tente predizer os semáforos na imagem
            pred.predict_image(add_noise=False)
        except ValueError as e:
            # Se nenhum semáforo for encontrado, imprima o erro
            print(f"Erro na imagem {filename}: {str(e)}")


image 1/1 c:\Users\Thiago\OneDrive\Documentos\Facul\6 Sexto semestre\TCC\TCC-CDIA\notebooks\..\data\frames\images\100_original.jpg: 384x640 6 cars, 2 traffic lights, 604.4ms
Speed: 2.0ms preprocess, 604.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



Semáforo 1 da imagem 100_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_100_original.jpg. Não foi salvo novamente.
Semáforo 2 da imagem 100_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_100_original(1).jpg. Não foi salvo novamente.
Carro 1 da imagem 100_original já existe em ../data/processed/outputs/carros/standard/car_100_original.jpg. Não foi salvo novamente.
Carro 2 da imagem 100_original já existe em ../data/processed/outputs/carros/standard/car_100_original(1).jpg. Não foi salvo novamente.
Carro 3 da imagem 100_original já existe em ../data/processed/outputs/carros/standard/car_100_original(2).jpg. Não foi salvo novamente.
Carro 4 da imagem 100_original já existe em ../data/processed/outputs/carros/standard/car_100_original(3).jpg. Não foi salvo novamente.
Carro 5 da imagem 100_original já existe em ../data/processed/outputs/carros/standard/car_100_original(4).jpg. Não foi salvo novamente.
Carro 6 da imagem 100_

image 1/1 c:\Users\Thiago\OneDrive\Documentos\Facul\6 Sexto semestre\TCC\TCC-CDIA\notebooks\..\data\frames\images\101_original.jpg: 384x640 10 persons, 4 cars, 3 traffic lights, 472.9ms
Speed: 2.0ms preprocess, 472.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



Semáforo 1 da imagem 101_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_101_original.jpg. Não foi salvo novamente.
Semáforo 2 da imagem 101_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_101_original(1).jpg. Não foi salvo novamente.
Semáforo 3 da imagem 101_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_101_original(2).jpg. Não foi salvo novamente.
Carro 1 da imagem 101_original já existe em ../data/processed/outputs/carros/standard/car_101_original.jpg. Não foi salvo novamente.
Carro 2 da imagem 101_original já existe em ../data/processed/outputs/carros/standard/car_101_original(1).jpg. Não foi salvo novamente.
Carro 3 da imagem 101_original já existe em ../data/processed/outputs/carros/standard/car_101_original(2).jpg. Não foi salvo novamente.
Carro 4 da imagem 101_original já existe em ../data/processed/outputs/carros/standard/car_101_original(3).jpg. Não foi salvo novamente.
Erro n

image 1/1 c:\Users\Thiago\OneDrive\Documentos\Facul\6 Sexto semestre\TCC\TCC-CDIA\notebooks\..\data\frames\images\102_original.jpg: 384x640 6 cars, 3 traffic lights, 508.6ms
Speed: 4.0ms preprocess, 508.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)



...

Semáforo 1 da imagem 98_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_98_original.jpg. Não foi salvo novamente.
Semáforo 2 da imagem 98_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_98_original(1).jpg. Não foi salvo novamente.
Carro 1 da imagem 98_original já existe em ../data/processed/outputs/carros/standard/car_98_original.jpg. Não foi salvo novamente.
Carro 2 da imagem 98_original já existe em ../data/processed/outputs/carros/standard/car_98_original(1).jpg. Não foi salvo novamente.
Carro 3 da imagem 98_original já existe em ../data/processed/outputs/carros/standard/car_98_original(2).jpg. Não foi salvo novamente.
Carro 4 da imagem 98_original já existe em ../data/processed/outputs/carros/standard/car_98_original(3).jpg. Não foi salvo novamente.
Carro 5 da imagem 98_original já existe em ../data/processed/outputs/carros/standard/car_98_original(4).jpg. Não foi salvo novamente.
Carro 6 da imagem 98_original já exi

image 1/1 c:\Users\Thiago\OneDrive\Documentos\Facul\6 Sexto semestre\TCC\TCC-CDIA\notebooks\..\data\frames\images\99_original.jpg: 384x640 4 cars, 3 traffic lights, 521.7ms
Speed: 3.0ms preprocess, 521.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



Semáforo 1 da imagem 99_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_99_original.jpg. Não foi salvo novamente.
Semáforo 2 da imagem 99_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_99_original(1).jpg. Não foi salvo novamente.
Semáforo 3 da imagem 99_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_99_original(2).jpg. Não foi salvo novamente.
Carro 1 da imagem 99_original já existe em ../data/processed/outputs/carros/standard/car_99_original.jpg. Não foi salvo novamente.
Carro 2 da imagem 99_original já existe em ../data/processed/outputs/carros/standard/car_99_original(1).jpg. Não foi salvo novamente.
Carro 3 da imagem 99_original já existe em ../data/processed/outputs/carros/standard/car_99_original(2).jpg. Não foi salvo novamente.
Carro 4 da imagem 99_original já existe em ../data/processed/outputs/carros/standard/car_99_original(3).jpg. Não foi salvo novamente.
Erro na imagem 99_or

image 1/1 c:\Users\Thiago\OneDrive\Documentos\Facul\6 Sexto semestre\TCC\TCC-CDIA\notebooks\..\data\frames\images\9_original.jpg: 384x640 9 cars, 1 bus, 1 truck, 487.8ms
Speed: 2.0ms preprocess, 487.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


Erro na imagem 9_original.jpg: Nenhum semáforo detectado na imagem.


In [10]:
import os
import cv2
import numpy as np

def add_noise_to_images(input_folder, output_folder, noise_pctg, suffix='noisy'):
    # Certifique-se de que a pasta de saída exista
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Listar arquivos na pasta de entrada
    image_files = os.listdir(input_folder)

    for image_file in image_files:
        if image_file.endswith('.jpg'):
            # Carregar a imagem original
            image_path = os.path.join(input_folder, image_file)
            image = cv2.imread(image_path)
            
            # Aplicar ruído à imagem inteira
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image.astype(np.float32) / 255.0
            noise = np.random.uniform(low=0.0, high=1.0, size=image.shape)
            noise_mask = np.random.binomial(1, noise_pctg, size=image.shape[:-1])
            noise_mask = np.expand_dims(noise_mask, axis=-1)
            noise_mask = np.repeat(noise_mask, 3, axis=-1)
            image = noise_mask * noise + (1 - noise_mask) * image
            image = (image * 255.0).astype(np.uint8)

            # Salvar a imagem com o novo nome
            noisy_image_file = f"{os.path.splitext(image_file)[0]}_{suffix}.jpg"
            noisy_image_path = os.path.join(output_folder, noisy_image_file)
            cv2.imwrite(noisy_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
            print(f"Imagem {image_file} com ruído aplicado e salva como {noisy_image_file}")

In [11]:
objetos = ['carros', 'pessoas', 'caminhao', 'semafaros', 'motos']
for i in range(len(objetos)):
    input_folder = f"../data/processed/outputs/{objetos[i]}/standard"
    output_folder = f"../data/processed/outputs/{objetos[i]}/noisy"
    noise_pctg = 0.3  # Porcentagem de ruído a ser aplicada
    add_noise_to_images(input_folder, output_folder, noise_pctg)

Imagem car_100_original(1).jpg com ruído aplicado e salva como car_100_original(1)_noisy.jpg
Imagem car_100_original(2).jpg com ruído aplicado e salva como car_100_original(2)_noisy.jpg
Imagem car_100_original(3).jpg com ruído aplicado e salva como car_100_original(3)_noisy.jpg
Imagem car_100_original(4).jpg com ruído aplicado e salva como car_100_original(4)_noisy.jpg
Imagem car_100_original(5).jpg com ruído aplicado e salva como car_100_original(5)_noisy.jpg
Imagem car_100_original.jpg com ruído aplicado e salva como car_100_original_noisy.jpg
Imagem car_101_original(1).jpg com ruído aplicado e salva como car_101_original(1)_noisy.jpg
Imagem car_101_original(2).jpg com ruído aplicado e salva como car_101_original(2)_noisy.jpg
Imagem car_101_original(3).jpg com ruído aplicado e salva como car_101_original(3)_noisy.jpg
Imagem car_101_original.jpg com ruído aplicado e salva como car_101_original_noisy.jpg
Imagem car_102_original(1).jpg com ruído aplicado e salva como car_102_original(1)

In [12]:
def contar_elementos_em_pasta(pasta):
    contador = 0
    for root, dirs, files in os.walk(pasta):
        contador += len(dirs)  # Conta subpastas
        contador += len(files)  # Conta arquivos
    return contador

pastas = ["../data/processed/outputs/caminhao/noisy/", "../data/processed/outputs/carros/noisy/", "../data/processed/outputs/motos/noisy/", "../data/processed/outputs/pessoas/noisy/", "../data/processed/outputs/semafaros/noisy/"]

for pasta in pastas:
    elementos = contar_elementos_em_pasta(pasta)
    print(f"Total de elementos na pasta {pasta}: {elementos}")


Total de elementos na pasta ../data/processed/outputs/caminhao/noisy/: 25
Total de elementos na pasta ../data/processed/outputs/carros/noisy/: 499
Total de elementos na pasta ../data/processed/outputs/motos/noisy/: 3
Total de elementos na pasta ../data/processed/outputs/pessoas/noisy/: 22
Total de elementos na pasta ../data/processed/outputs/semafaros/noisy/: 151
